# AI Drag and Drop

In [ ]:
#Importing libraries 
import mediapipe
import cv2
from cvzone.HandTrackingModule import HandDetector
import cvzone
import os 

#calling hand detecotr module for detection
detector = HandDetector(detectionCon=0.8)

#Folder path
path ="ImagesMix"
myList = os.listdir(path)

#class help to store all the attribute of images
class DragImg():
    def __init__(self,path,posOrigin,imgType):                        #Storing path, origin position, and image type
        self.posOrigin = posOrigin                   
        self.imgType = imgType
        self.path = path
        if self.imgType =="png":                                      #checking if img is png or not 
            self.img = cv2.imread(self.path,cv2.IMREAD_UNCHANGED)
        else:                                                         #according to that we will reflect img on our screen
            self.img = cv2.imread(self.path)
            
        self.size = self.img.shape[:2]                                #Fetching image height and width
        
        
    def update(self,cursor):                                          #Updating cursor whenever we click 
        ox,oy = self.posOrigin                                        #fetching position
        h,w = self.size
        #checking if it in region 
        if ox<cursor[0]<ox+w and oy<cursor[1]<oy+h:                   #checking if image is under certain condition
            self.posOrigin = cursor[0]-w//2,cursor[1]-h//2            #Updating image to the center of fore finger
    
    
#Storing list of images with respect ot class
listImg =[]                              
for x,pathImg in enumerate(myList):
    if 'png' in pathImg:
        imgType ='png'
    else:
        imgType ='jpg'
        
    listImg.append(DragImg(f'{path}/{pathImg}',[50+x*300,50],imgType))   #passing all data to class
    
    
#Opeing camera and seting height and width
cam = cv2.VideoCapture(0)
cam.set(3,1280)
cam.set(4,720)

#While loop help to see clicked images and trained model setup
while True:
    _,img = cam.read()
    if _:
        img = cv2.flip(img,1)
        hand,img=detector.findHands(img,flipType=False)           #Finding hands and drawed img 
        
        if hand:                                                  #if we find hand 
            lmList=hand[0]['lmList']                              #fetch all the landmark of first hand apper in frame
            #checking if clicked or not
            length,info,img=detector.findDistance(lmList[8],lmList[12],img)    #fetching distance which help to shift img
            if length <40:                                        #if dis is bilow 40 then we will perform shifting task
                cursor = lmList[8]                                #Fetching for fingure data
                for imgObject in listImg:                         
                    imgObject.update(cursor)                      #this function will check if forfinger is
                                                                  #present in given location or not
                
                    
            
#Drawing all iamges 
        for imgObject in listImg:                                  #This loop will help to set images om screen
            
            h,w= imgObject.size
            ox,oy = imgObject.posOrigin
            if imgObject.imgType =='png':                          
                img = cvzone.overlayPNG(img,imgObject.img,[ox,oy])
            else:
                # Draw for JPG images
                img[oy:oy+h,ox:ox+w]= imgObject.img
        cv2.imshow("Img",img)
        if cv2.waitKey(1) ==13:
            break    
    else:
        print("No image")
        break
cam.release()
cv2.destroyAllWindows()